# Import Library

In [1]:
import os

from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_text_splitters import LatexTextSplitter, RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Retrieval

In [2]:
# Inisialisasi model embedding Ollama
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Splitter umum (untuk teks biasa)
default_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Splitter khusus LaTeX
latex_splitter = LatexTextSplitter(chunk_size=1000, chunk_overlap=200)

In [3]:
# Folder berisi PDF
pdf_folder = "./documents"

# Tempat simpan vektor
persist_dir = "./chroma_db"

In [4]:
all_docs = []

# Loop semua file PDF dalam folder
for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):
        path = os.path.join(pdf_folder, filename)
        print(f"Memproses: {filename}")

        # Load PDF
        loader = PyPDFLoader(path)
        docs = loader.load()

        # Bersihkan teks sebelum split
        for d in docs:
            text = str(d.page_content).encode("utf-8", "ignore").decode()

            if any(
                token in text for token in ["\\begin{", "\\end{", "$", "\\frac", "\\sum", "\\int"]
            ):
                splits = latex_splitter.split_text(text)
            else:
                splits = default_splitter.split_text(text)

            for s in splits:
                all_docs.append(Document(page_content=s, metadata=d.metadata))

Memproses: 01_02Sim1R2 Dasar-Dasar Pemodelan Sistem.pdf
Memproses: 03&04_Latihan Var. Acak dan Probabilitas.pdf
Memproses: 02_Bab2_elemen_probabilitas.pdf
Memproses: 06_Variabel Acak Binomial dan Poisson (new).pdf
Memproses: 03_Latihan2Probabilitas&VarAcak.pdf


In [6]:
# Simpan embedding ke Chroma
print("Membuat embedding dan menyimpan ke ChromaDB...")
db = Chroma.from_documents(all_docs, embedding=embeddings, persist_directory=persist_dir)
print("Selesai! Semua PDF telah di-embed dan disimpan di:", persist_dir)

Membuat embedding dan menyimpan ke ChromaDB...
Selesai! Semua PDF telah di-embed dan disimpan di: ./chroma_db


# LLM (llama3)

In [7]:
# Load database
embeddings = OllamaEmbeddings(model="nomic-embed-text")
persist_dir = "./chroma_db"

db = Chroma(persist_directory=persist_dir, embedding_function=embeddings)
retriever = db.as_retriever(search_kwargs={"k": 3})

In [8]:
# Model Ollama lokal
llm = OllamaLLM(model="llama3")

In [9]:
# Template prompt
prompt = ChatPromptTemplate.from_template("""
Gunakan informasi berikut untuk menjawab pertanyaan dengan akurat dan jelas.
Jika tidak tahu jawabannya, katakan tidak tahu.

Konteks:
{context}

Pertanyaan:
{question}
""")

In [13]:
# Bangun pipeline RAG gaya baru (LCEL)
rag_chain = (
    {
        "context": retriever,
        "question": lambda x: x,
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
def chat():
    q = input("Pertanyaan: ").strip()
    ans = rag_chain.invoke(q)

    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ans}")

In [20]:
chat()

Pertanyaan: apakah sistem itu?
Jawaban:
Menurut informasi dokumentasi, sistem itu adalah kumpulan elemen yang bekerja bersama untuk mencapai tujuan yang diharapkan. Informasi ini tercantum pada dokumen dengan id '56fe2982-6788-4119-9619-130fa25c09dd' pada halaman 4.
